In [67]:
import re 

#double damma, double fatha, double kasera, damma, fatha, kasera, sukoon, shadd
TASHKEEL_SET = {'ٌ', 'ً', 'ٍ', 'ُ', 'َ', 'ِ', 'ْ', 'ٌّ', 'ّ'}
DIACRITICS_REGEX = re.compile('|'.join(TASHKEEL_SET))
def remove_tashkeel(data):
    return re.sub(DIACRITICS_REGEX, '', data)

print(remove_tashkeel('الْأَلْبَاب'))

الألباب


In [68]:
def one_hot_encode(indices, size):
    return [[1 if i == elem else 0 for i in range(size)] for elem in indices]

print(one_hot_encode([0, 1, 2],10) )

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]


In [69]:
import re

def chunk_text(text, chunk_size):
    chunks = []
    words = re.findall(r'\S+', text)

    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += f"{word} "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = f"{word} "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return list(filter(None, chunks))

def tokenize_lines(data_raw):
    # Define a pattern to match specific punctuation marks
    punctuation_pattern1 = r'([.,:;؛)\]}»،])'
    punctuation_pattern2 = r'([(\[{«])'
    tokenized_lines = []

    for line in data_raw:
        # Replace matched punctuation marks with the same followed by a line break
        splitted_line = re.sub(punctuation_pattern1, r'\1\n', line)
        splitted_line = re.sub(punctuation_pattern2, r'\n\1', splitted_line)

        # Further split the splitted line into substrings based on line breaks
        for sub_line in splitted_line.split('\n'):
            cleaned_sub_line = remove_tashkeel(sub_line).strip()
            if 0 < len(cleaned_sub_line) <= 500:
                tokenized_lines.append(sub_line.strip())

            elif len(cleaned_sub_line) > 500:
                tokenized_lines.extend(chunk_text(sub_line.strip(), 500))
    
    return tokenized_lines


In [70]:
val_raw = None
with open('../data/val.txt', encoding='utf-8') as file:
    val_raw = file.readlines()
print('Validation examples (raw):', len(val_raw))

val_split = tokenize_lines(val_raw)

file_path = "../data/val_split_output.txt"

# Write each list item to a new line in the text file
with open(file_path, 'w', encoding='utf-8') as file:
    for item in val_split:
        file.write(f"{item}\n")

print(f"Val split saved to {file_path}")


Validation examples (raw): 2500
Val split saved to ../data/val_split_output.txt
